In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50 
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Lambda, Input, AveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import random
#import cv2
import os

In [2]:
covid_dir=os.path.join("Database/COVID19")
norm_dir=os.path.join("Database/NORMAL")
pneu_dir=os.path.join("Database/Viral_Pneumonia")


print('total Covid images:', len(os.listdir(covid_dir)))
print('total Normal images:', len(os.listdir(norm_dir)))
print('total Viral_Pneumonia:', len(os.listdir(pneu_dir)))

total Covid images: 1327
total Normal images: 1341
total Viral_Pneumonia: 1463


In [3]:
img_width, img_height=(224,224)
batch_size=128

data_dir = "Database/"

# Rescale images 
datagen= ImageDataGenerator(rescale=1/255,validation_split=.3,rotation_range=20,
                           shear_range=.2,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,
                           preprocessing_function=preprocess_input)

# Flow training images in batches of 128 using train_data 
train_generator= datagen.flow_from_directory(
                data_dir,
                target_size=(img_width,img_height),
                batch_size=batch_size,
                subset="training",
                class_mode="categorical", 
                classes= ["COVID19", "NORMAL","Viral_Pneumonia"],
                shuffle=True, seed=30)


datagen2=ImageDataGenerator(rescale=1/255,validation_split=.3,
                           preprocessing_function=preprocess_input)

test_generator=datagen2.flow_from_directory(
                data_dir,
                target_size=(img_width,img_height),
                batch_size=batch_size,
                classes= ["COVID19", "NORMAL","Viral_Pneumonia"],
                class_mode="categorical", subset="validation", shuffle=True, seed=30)

Found 2893 images belonging to 3 classes.
Found 1238 images belonging to 3 classes.


## Transfer Learning Model- Resnet

In [5]:
classifier = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

# x=classifier.output
# x=AveragePooling2D()(x)
# x=Dense(64, activation="relu")(x)
# predictions=Dense(3, activation="softmax")(x)
# model=Model(inputs=classifier.input,outputs=predictions)

x=Flatten()(classifier.output)
x=Dense(3, activation="softmax")(x)
model=Model(inputs=classifier.input,outputs=x)


for layer in classifier.layers:
    layer.trainable=False
    
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=["accuracy"])

In [ ]:
total_sample= train_generator.n
batch_size=128


history= model.fit(train_generator,epochs=30, validation_data=test_generator)

Epoch 1/30
23/23 [==============================] - 196s 9s/step - loss: 0.6709 - accuracy: 0.7176 - val_loss: 0.6946 - val_accuracy: 0.6777
Epoch 2/30
23/23 [==============================] - 196s 9s/step - loss: 0.7004 - accuracy: 0.6799 - val_loss: 0.4786 - val_accuracy: 0.8263
Epoch 3/30
23/23 [==============================] - 191s 8s/step - loss: 0.6360 - accuracy: 0.7269 - val_loss: 0.5913 - val_accuracy: 0.7585
Epoch 4/30
23/23 [==============================] - 190s 8s/step - loss: 0.5879 - accuracy: 0.7632 - val_loss: 0.4974 - val_accuracy: 0.8021
Epoch 5/30
23/23 [==============================] - 233s 10s/step - loss: 0.6051 - accuracy: 0.7363 - val_loss: 0.4967 - val_accuracy: 0.7690
Epoch 6/30
23/23 [==============================] - 192s 8s/step - loss: 0.6654 - accuracy: 0.7031 - val_loss: 0.5400 - val_accuracy: 0.7779
Epoch 7/30
23/23 [==============================] - 191s 8s/step - loss: 0.5542 - accuracy: 0.7712 - val_loss: 0.5878 - val_accuracy: 0.7536
Epoch 8/30
2

In [ ]:
# Save Model
history.save("Database/Saved_Models/ResNet_model.h5")

In [ ]:
test_loss, test_acc= model.evaluate(test_generator)
print("\n Test Accuracy:", test_acc)